In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
from pprint import pprint

# Specify the path to your uploaded JSON file
json_file_path = '/content/drive/MyDrive/Research/ICML_2023_Raw/DataFile/submission_details_wid.jsonl'

# Open the JSON file and load the data
with open(json_file_path, 'r') as file:
    data = json.load(file)

filtered_json_data = []
# Iterate through each dictionary in the data
for obj in data:
    # Check if the dictionary has the key 'invitation' and if its value contains 'Submission6522'
    if '4abacaafefc7983e199c198c7fbbc9c02ed2d97f65c1bc5c5f6f1b719a88ef76' in obj['submission_id']:
        filtered_json_data.append(obj)


word_counts = 0
reply_counts = 0
for i in range(len(filtered_json_data)):
      #if '/Official_Review' in Submission[n][i]['invitations'][0]:
        #word_counts += len(Submission[n][i]['content'].split())
    if '/Rebuttal' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['rebuttal']['value'].split())
      reply_counts += 1
    if '/Official_Comment' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['comment']['value'].split())
      reply_counts += 1



Oral_rebuttal = {}
Poster_rebuttal = {}
Reject_rebuttal = {}
Withdrawal_desk_rebuttal = {}


for i, sub_dict in enumerate(filtered_json_data):
    if '/Withdrawal' in sub_dict['invitations'][0] or '/Desk_Rejection' in sub_dict['invitations'][0]:
        print(sub_dict['submission_id'], ":g!")
        break  # Stop inner loop if condition is met

else:
  if filtered_json_data[-1]['content']['decision']['value'] == 'Reject':
    Reject_rebuttal[filtered_json_data[-1]['forum']] = (word_counts, reply_counts)
  if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Poster)':
    Poster_rebuttal[filtered_json_data[-1]['forum']] = (word_counts, reply_counts)
  if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Oral & Poster)':
    Oral_rebuttal[filtered_json_data[-1]['forum']] = (word_counts, reply_counts)


{'id': 'RMXQgVg04R', 'number': 1, 'cdate': 1676941946959, 'tcdate': 1676941946959, 'mdate': 1678801068784, 'tmdate': 1678801068784, 'forum': 'Lt0lAQtiI9', 'replyto': 'Lt0lAQtiI9', 'content': {'summary': {'value': 'This paper proposes a retrieval-augmented model for image captioning. The retrieval model uses CLIP image embedding to find similar images in the image-text database. The captioning model is a combination of the RETRO model and the Flamingo model architecture. The paper also proposes a filtering strategy to improve pre-training performance. The proposed model achieves improved performance for zero-shot/few-shot image captioning.'}, 'strengths_and_weaknesses': {'value': "Strengths:\n- Retrieval-augmented image captioning is an interesting direction to explore.\n- The proposed filtering strategy is useful to prevent shortcut solutions.\n\nWeakness:\n- The paper could have gone deeper in its analysis on the pros and cons of retrieval-augmented image captioning. There are several

In [2]:
import json
from pprint import pprint
from sklearn.isotonic import isotonic_regression
import cvxpy as cp
import numpy as np
import csv, os
from scipy import stats
from sklearn import metrics
import itertools
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statistics

# Specify the path to your uploaded JSON file
json_file_path = '/content/drive/MyDrive/Research/ICML_2023_Raw/DataFile/submission_details_wid.jsonl'

# Open the JSON file and load the data
with open(json_file_path, 'r') as file:
    data = json.load(file)


# Load CSV file into a pandas DataFrame
df = pd.read_csv(r'/content/drive/MyDrive/Research/ICML_2023_Result/proxy_score.csv')
df = df.drop_duplicates(['submission_idx', 'author_idx'])


# For each author, list its ranked submissions according to its ranking

author_submission_rank_4 = {}

# Extract the unique authors from the DataFrame
authors = df['author_id'].unique()

# Loop through every submission
for author in authors:

    author_submission_rank_4[author] = []

    # Extract the rating as sets
    submissions = df[df['author_id'] == author]['submission_id'].tolist()

    for i in range(len(submissions)):

        rank = df[(df['submission_id'] == submissions[i]) & (df['author_id'] == author)]['rank'].tolist()
        rank = rank[0]

        # Add the ratings to the results dictionary
        author_submission_rank_4[author].append((submissions[i], rank))

def sort_submissions(author_submission_rank_4):
    for author in author_submission_rank_4:
        author_submission_rank_4[author].sort(key=lambda x: x[1], reverse=False)
    return author_submission_rank_4

author_submission_rank_4 = sort_submissions(author_submission_rank_4)


High_submission_list = []
Low_submission_list = []
for author in author_submission_rank_4:
    # for i in range(len(author_submission_rank_4[author]) - 1):
        High_submission_list.append(author_submission_rank_4[author][0][0] )
        Low_submission_list.append(author_submission_rank_4[author][-1][0] )



High_Oral_rebuttal = {}
High_Poster_rebuttal = {}
High_Reject_rebuttal = {}
High_Withdrawn_rebuttal = {}

for submission in High_submission_list:
  filtered_json_data = []
  # Iterate through each dictionary in the data
  for obj in data:
      # Check if the dictionary has the key 'invitation' and if its value contains 'Submission6522'
      if submission in obj['submission_id']:
          filtered_json_data.append(obj)

  word_counts = 0
  reply_counts = 0
  for i in range(len(filtered_json_data)):
      #if '/Official_Review' in filtered_json_data[i]['invitations'][0]:
        #word_counts += len(filtered_json_data[i]['content'].split())
    if '/Rebuttal' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['rebuttal']['value'].split())
      reply_counts += 1
    if '/Official_Comment' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['comment']['value'].split())
      reply_counts += 1

  for i, sub_dict in enumerate(filtered_json_data):
      if '/Withdrawal' in sub_dict['invitations'][0] or '/Desk_Rejection' in sub_dict['invitations'][0]:
          # print(sub_dict['submission_id'], ":g!")
          break  # Stop inner loop if condition is met

  else:
    if filtered_json_data[-1]['content']['decision']['value'] == 'Reject':
      High_Reject_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Poster)':
      High_Poster_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Oral & Poster)':
      High_Oral_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)


High_Oral = []
High_Poster = []
High_Reject = []


for key in High_Oral_rebuttal:
  High_Oral.append( int(High_Oral_rebuttal[key][0]) )
for key in High_Poster_rebuttal:
  High_Poster.append( int(High_Poster_rebuttal[key][0]) )
for key in High_Reject_rebuttal:
  High_Reject.append( int(High_Reject_rebuttal[key][0]) )



Low_Oral_rebuttal = {}
Low_Poster_rebuttal = {}
Low_Reject_rebuttal = {}
Low_Withdrawn_rebuttal = {}

for submission in Low_submission_list:
  filtered_json_data = []
  # Iterate through each dictionary in the data
  for obj in data:
      # Check if the dictionary has the key 'invitation' and if its value contains 'Submission6522'
      if submission in obj['submission_id']:
          filtered_json_data.append(obj)

  word_counts = 0
  reply_counts = 0
  for i in range(len(filtered_json_data)):
      #if '/Official_Review' in filtered_json_data[i]['invitations'][0]:
        #word_counts += len(filtered_json_data[i]['content'].split())
    if '/Rebuttal' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['rebuttal']['value'].split())
      reply_counts += 1
    if '/Official_Comment' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['comment']['value'].split())
      reply_counts += 1

  for i, sub_dict in enumerate(filtered_json_data):
      if '/Withdrawal' in sub_dict['invitations'][0] or '/Desk_Rejection' in sub_dict['invitations'][0]:
          break  # Stop inner loop if condition is met

  else:
    if filtered_json_data[-1]['content']['decision']['value'] == 'Reject':
      Low_Reject_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Poster)':
      Low_Poster_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Oral & Poster)':
      Low_Oral_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)


Low_Oral = []
Low_Poster = []
Low_Reject = []


for key in Low_Oral_rebuttal:
  Low_Oral.append( int(Low_Oral_rebuttal[key][0]) )
for key in Low_Poster_rebuttal:
  Low_Poster.append( int(Low_Poster_rebuttal[key][0]) )
for key in Low_Reject_rebuttal:
  Low_Reject.append( int(Low_Reject_rebuttal[key][0]) )



High_Accept = High_Oral + High_Poster
Low_Accept = Low_Oral + Low_Poster

High_Reject = High_Reject
Low_Reject = Low_Reject


# Perform the unpaired t-test
t_statistic, p_value = stats.ttest_ind(High_Accept, Low_Accept, alternative='greater')

# Print the results
print("Accepted T-statistic:", t_statistic)
print("Accepted P-value:", p_value)

# Perform the unpaired t-test
t_statistic, p_value = stats.ttest_ind(High_Reject, Low_Reject, alternative='greater')

# Print the results
print("Rejected T-statistic:", t_statistic)
print("Rejected P-value:", p_value)


print(np.mean(High_Accept))
print(np.mean(Low_Accept))
print(np.mean(High_Reject))
print(np.mean(Low_Reject))


Accepted T-statistic: 1.2587205133054775
Accepted P-value: 0.10426332261770718
Rejected T-statistic: 1.5580958097122408
Rejected P-value: 0.059730333785854864
256.0846681922197
229.21428571428572
314.3530377668309
279.50911854103344


In [5]:
import json
from pprint import pprint
from sklearn.isotonic import isotonic_regression
import cvxpy as cp
import numpy as np
import csv, os
from scipy import stats
from sklearn import metrics
import itertools
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statistics

# Specify the path to your uploaded JSON file
json_file_path = '/content/drive/MyDrive/Research/ICML_2023_Raw/DataFile/submission_details_wid.jsonl'

# Open the JSON file and load the data
with open(json_file_path, 'r') as file:
    data = json.load(file)


# Load CSV file into a pandas DataFrame
df = pd.read_csv(r'/content/drive/MyDrive/Research/ICML_2023_Result/proxy_score.csv')
df = df.drop_duplicates(['submission_idx', 'author_idx'])


# For each author, list its ranked submissions according to its ranking

author_submission_rank_4 = {}

# Extract the unique authors from the DataFrame
authors = df['author_id'].unique()

# Loop through every submission
for author in authors:

    author_submission_rank_4[author] = []

    # Extract the rating as sets
    submissions = df[df['author_id'] == author]['submission_id'].tolist()

    for i in range(len(submissions)):

        rank = df[(df['submission_id'] == submissions[i]) & (df['author_id'] == author)]['rank'].tolist()
        rank = rank[0]

        # Add the ratings to the results dictionary
        author_submission_rank_4[author].append((submissions[i], rank))

def sort_submissions(author_submission_rank_4):
    for author in author_submission_rank_4:
        author_submission_rank_4[author].sort(key=lambda x: x[1], reverse=False)
    return author_submission_rank_4

author_submission_rank_4 = sort_submissions(author_submission_rank_4)


High_submission_list = []
Low_submission_list = []
for author in author_submission_rank_4:
    # for i in range(len(author_submission_rank_4[author]) - 1):
        High_submission_list.append(author_submission_rank_4[author][0][0] )
        Low_submission_list.append(author_submission_rank_4[author][-1][0] )

High_Oral_rebuttal = {}
High_Poster_rebuttal = {}
High_Reject_rebuttal = {}

for submission in High_submission_list:
  filtered_json_data = []
  # Iterate through each dictionary in the data
  for obj in data:
      # Check if the dictionary has the key 'invitation' and if its value contains 'Submission6522'
      if submission in obj['submission_id']:
          filtered_json_data.append(obj)

  word_counts = 0
  reply_counts = 0
  for i in range(len(filtered_json_data)):
      #if '/Official_Review' in filtered_json_data[i]['invitations'][0]:
        #word_counts += len(filtered_json_data[i]['content'].split())
    if '/Rebuttal' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['rebuttal']['value'].split())
      reply_counts += 1
    if '/Official_Comment' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['comment']['value'].split())
      reply_counts += 1

  for i, sub_dict in enumerate(filtered_json_data):
      if '/Withdrawal' in sub_dict['invitations'][0] or '/Desk_Rejection' in sub_dict['invitations'][0]:
          break  # Stop inner loop if condition is met

  else:
    if filtered_json_data[-1]['content']['decision']['value'] == 'Reject':
      High_Reject_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Poster)':
      High_Poster_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Oral & Poster)':
      High_Oral_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)


High_Oral = []
High_Poster = []
High_Reject = []

for key in High_Oral_rebuttal:
  High_Oral.append( int(High_Oral_rebuttal[key][1]) )
for key in High_Poster_rebuttal:
  High_Poster.append( int(High_Poster_rebuttal[key][1]) )
for key in High_Reject_rebuttal:
  High_Reject.append( int(High_Reject_rebuttal[key][1]) )


Low_Oral_rebuttal = {}
Low_Poster_rebuttal = {}
Low_Reject_rebuttal = {}

for submission in Low_submission_list:
  filtered_json_data = []
  # Iterate through each dictionary in the data
  for obj in data:
      # Check if the dictionary has the key 'invitation' and if its value contains 'Submission6522'
      if submission in obj['submission_id']:
          filtered_json_data.append(obj)

  word_counts = 0
  reply_counts = 0
  for i in range(len(filtered_json_data)):
    if '/Rebuttal' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['rebuttal']['value'].split())
      reply_counts += 1
    if '/Official_Comment' in filtered_json_data[i]['invitations'][0]:
      word_counts += len(filtered_json_data[i]['content']['comment']['value'].split())
      reply_counts += 1

  for i, sub_dict in enumerate(filtered_json_data):
      if '/Withdrawal' in sub_dict['invitations'][0] or '/Desk_Rejection' in sub_dict['invitations'][0]:
          break  # Stop inner loop if condition is met

  else:
    if filtered_json_data[-1]['content']['decision']['value'] == 'Reject':
      Low_Reject_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Poster)':
      Low_Poster_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)
    if filtered_json_data[-1]['content']['decision']['value'] == 'Accept (Oral & Poster)':
      Low_Oral_rebuttal[filtered_json_data[-1]['submission_id']] = (word_counts, reply_counts)


Low_Oral = []
Low_Poster = []
Low_Reject = []

for key in Low_Oral_rebuttal:
  Low_Oral.append( int(Low_Oral_rebuttal[key][1]) )
for key in Low_Poster_rebuttal:
  Low_Poster.append( int(Low_Poster_rebuttal[key][1]) )
for key in Low_Reject_rebuttal:
  Low_Reject.append( int(Low_Reject_rebuttal[key][1]) )

High_Accept = High_Oral + High_Poster
Low_Accept = Low_Oral + Low_Poster


# Perform the unpaired t-test
t_statistic, p_value = stats.ttest_ind(High_Accept, Low_Accept, alternative='greater')

# Print the results
print("Accepted T-statistic:", t_statistic)
print("Accepted P-value:", p_value)

# Perform the unpaired t-test
t_statistic, p_value = stats.ttest_ind(High_Reject, Low_Reject, alternative='greater')

# Print the results
print("Rejected T-statistic:", t_statistic)
print("Rejected P-value:", p_value)


print(np.mean(High_Accept))
print(np.mean(Low_Accept))
print(np.mean(High_Reject))
print(np.mean(Low_Reject))


Accepted T-statistic: 1.6347032389018232
Accepted P-value: 0.051267423738227826
Rejected T-statistic: 1.5492497810007824
Rejected P-value: 0.06078590746782445
1.794050343249428
1.6331168831168832
1.9507389162561577
1.8145896656534954
